In [38]:
%load_ext autoreload
%autoreload 2
import pandas as pd
import numpy as np
from credentials import Credentials as cr
import os

In [ ]:
abra = pd.read_csv(cr.PATH+'2019.txt', sep='|')
kadabra = abra.info()
print (kadabra)

In [36]:
df = abra.filter(['Code postal','Nombre pieces principales', 'Surface terrain', 'Surface Carrez du 1er lot', 'Surface Carrez du 2eme lot', 'Surface Carrez du 3eme lot', 'Surface Carrez du 4eme lot', 'Surface Carrez du 5eme lot', 'Valeur fonciere'])
df.head()

,Code postal,Nombre pieces principales,Surface terrain,Surface Carrez du 1er lot,Surface Carrez du 2eme lot,Surface Carrez du 3eme lot,Surface Carrez du 4eme lot,Surface Carrez du 5eme lot,Valeur fonciere
0,1000.0,1.0,NaN,"19,27",NaN,NaN,NaN,NaN,"37220,00"
1,1000.0,3.0,NaN,NaN,"61,51",NaN,NaN,NaN,"185100,00"
2,1000.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,"185100,00"
3,1160.0,4.0,940.0,NaN,NaN,NaN,NaN,NaN,"209000,00"
4,1370.0,5.0,490.0,NaN,NaN,NaN,NaN,NaN,"134900,00"


In [37]:
#dropdup = df.drop_duplicates()
final = df.info()

&lt;class &#39;pandas.core.frame.DataFrame&#39;&gt;
RangeIndex: 2535791 entries, 0 to 2535790
Data columns (total 9 columns):
 #   Column                      Dtype  
---  ------                      -----  
 0   Code postal                 float64
 1   Nombre pieces principales   float64
 2   Surface terrain             float64
 3   Surface Carrez du 1er lot   object 
 4   Surface Carrez du 2eme lot  object 
 5   Surface Carrez du 3eme lot  object 
 6   Surface Carrez du 4eme lot  object 
 7   Surface Carrez du 5eme lot  object 
 8   Valeur fonciere             object 
dtypes: float64(3), object(6)
memory usage: 174.1+ MB
